In [1]:
import riak

client = riak.RiakClient()

In [2]:
client.is_alive()

True

http://docs.basho.com/riak/kv/2.2.3/developing/data-types/

http://docs.basho.com/riak/kv/2.2.3/developing/data-types/counters/#create-a-counter

## Create a counter

In [6]:
bucket = client.bucket_type('counters').bucket('traffic_tickets')

In [7]:
counter = bucket.new('flights1')

## Increment a counter

In [38]:
counter.increment()

# Updates are staged locally and have to be explicitly sent to Riak
# using the store() method.
counter.store()

## Increment a counter with more that one

In [39]:
counter.increment(5)
counter.store()

## Retrieve a counter value

In [40]:
# To fetch the value stored on the server, use the call below. Note
# that this will clear any changes to the counter that have not yet been
# sent to Riak
counter.reload()

print "Value:", counter.value

Value: 18


In [42]:
## Decrement a counter value

In [43]:
counter.decrement()

# Just like incrementing, you can also decrement by more than one, e.g.:
counter.decrement(3)
counter.store()

## Retrieve a counter value with curl

In [46]:
!curl http://localhost:8098/types/counters/buckets/traffic_tickets/datatypes/flights1

{"type":"counter","value":14}